In [43]:
from langchain.chat_models.gigachat import GigaChat
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from chromadb.config import Settings
from langchain.vectorstores import Chroma
from langchain_community.embeddings.gigachat import GigaChatEmbeddings

In [44]:
llm = GigaChat(credentials="NWM1MDQxOWYtNzk0Yi00NzI4LTkxNTctNTY5YWFjYzcyM2MxOjc3MGVmZTdiLTYzNzUtNGE3OC1iN2Y0LTU5ODNiNWJjYWZjNQ==", verify_ssl_certs=False)

In [45]:
from langchain.schema import HumanMessage

question = "У меня на правой ягодице пятно зеленого цвета."
question = f"Выдели ключевые слова, необходимые для поиска медицинских документов для ответа на вопросы по этой теме: '{question}'. Перечисли слова через запятую."
text = llm([HumanMessage(content=question)]).content[0:200]


In [46]:
import re
pattern = r'"(.*?)"'
matches = re.findall(pattern, text)
print(matches)

['пятно зеленого цвета', 'ягодица правая']


In [47]:
import urllib
import requests

def get_links(keyword: str, limit: int = 3) -> tuple[list, int]:
    f = {"q":keyword,"type":"clinical", "offset":"0", "limit":f"{limit}"}
    url = "https://demo.onco-reg.ru/docs/api?" + urllib.parse.urlencode(f)
    answer = requests.get(url).json()
    number_of_pdf: int = int(answer['result']['total'])
    return [doc['link'] for doc in answer['result']['documents']], number_of_pdf

In [48]:
links = set()
for keyword in matches:
    x, y = get_links(keyword=keyword)
    for link in x:
        links.add(link)
        
links

{'https://demo.oncor.pro/docs/oncor-docs/clinical-guidelines/other/uvealnaya_melanoma.pdf',
 'https://demo.oncor.pro/docs/oncor-docs/clinical-guidelines/procto/rak_analnogo_kanala.pdf',
 'https://demo.oncor.pro/docs/oncor-docs/clinical-guidelines/procto/rak_analnogo_kanala_2024.pdf',
 'https://demo.oncor.pro/docs/oncor-docs/clinical-guidelines/skin/melanoma_kozhi_deti.pdf'}

In [49]:
from pdfminer.high_level import extract_text
import io

PDFs = '\n'.join([extract_text(io.BytesIO(requests.get(link).content)) for link in links])

In [50]:
with open("tmp.txt", 'w+', encoding='utf-8') as file:
    file.write(PDFs)